<h3 style="text-align: center;">ESCUELA TÉCNICA SUPERIOR DE INGENIERÍA INFORMÁTICA</h3>
<h3 style="text-align: center;">UNIVERSIDAD DE SEVILLA</h3>
<h3 style="text-align: center;">MATEMÁTICA APLICADA A SISTEMAS DE INFORMACIÓN</h3>
<h3 style="text-align: center;">Tercera práctica de laboratorio</h3>
<h3>Programación lineal: Conceptos básicos</h3>
<p>En esta práctica trataremos la Programación Lineal y el método del Símplex. Hemos estudido cómo la región factible de un problema de programación lineal es un convexo, concretamente un polítopo. Gracias al Teorema de representación si $S$ es la región factible de nuestro problema descrita en formato estándar, por lo tanto $S=  \{ \mathbf{x} \in {\mathbb R}^n: \,   A \, \mathbf{x} = \mathbf{b},\, \mathbf{x} \ge \mathbf{0} \}$, con $A$ matriz $m \times n$ de rango $m$. <br />Sean $\mathbf{x}^1, \ldots, \mathbf{x}^k$ los puntos extremos de $S$, y $\vec{\mathbf{d}}^1, \ldots, \vec{\mathbf{d}}^q$ las  direcciones extremas de $S$</p>
<p>$$ \mathbf{x} \in S \Longleftrightarrow \mathbf{x} = \sum_{j=1}^k \lambda_j \, \mathbf{x}^j \, +\, \sum_{j=1}^q \, \mu_j \, \vec{\mathbf{d}}^j,\; \mbox{con} \; \lambda_j \ge 0, \, \sum_{j=1}^k \lambda_j=1, \; \mu_j \ge 0$$</p>
<p>Partamos del mismo ejemplo que hemos visto en clase de teoría:</p>
<p>$$\left. \begin{array}{rl}\min & Z(x_1, x_2) \\ \mbox{s.t.} & - x_1 + x_2 \leq 2 \\&  - x_1 + 2 x_2 \leq 6 \\&  x_1 \geq 0, x_2 \geq 0 \end{array} \right\}$$</p>
<p>En formato estándar:</p>
<p>$$\left. \begin{array}{rl}\min & Z(x_1, x_2) \\ \mbox{s.t.} & - x_1 + x_2 +x_3\hspace{.77cm}=2 \\&  - x_1 + 2 x_2 \hspace{.6cm}+x_4= 6 \\&  x_j \geq 0, \; j=1,\ldots,4 \end{array} \right\}$$</p>
<p>Estamos interesados en calcular los puntos extremos de nuestra región factible, para ellos necesitaremos ciertas herramientas del Álgebra lineal, si introducimos las matriz de coeficientes $A$ y el vector $b$:</p>

In [114]:
A=matrix([[-1,1,1,0],[-1,2,0,1]])
b=vector([2,6])

In [115]:
A;show(A)

[-1  1  1  0]
[-1  2  0  1]

<p>Si estamos interesados en el rango de $A$, número de filas o columnas</p>

In [116]:
rank(A);A.nrows();A.ncols()

4

<p><span id="cell_outer_23"><span id="cell_outer_14">Para resolver <span><span><span>el sistema de ecuaciones lineales Ax</span><span style="position: relative; margin-left: 0.277em;"><span>=</span></span><span style="position: relative; margin-left: 0.277em;"><span>b</span></span></span></span> disponemos, al menos, de dos formas: </span></span></p>

In [117]:
A.solve_right(b)

(2, 4, 0, 0)

In [66]:
A\b

(2, 4, 0, 0)

<p>Nuestro sistema tiene infinitas soluciones pero SAGE sólo nos proporciona una, que es además una solución básica factible con base $\{1,2\}$. Estamos interesados en calcular todos los puntos extremos, como hemos visto en teoría, para ello nos basta resolver sistemas lineales donde la matriz de coeficientes sea submatriz de $A$ con rango máximo. Veamos en primer lugar cómo seleccionar una submatriz de $A$. Teniendo en cuenta que SAGE comienza a contar en el 0, si queremos tomar las dos filas de $A$ (primera y segunda fila, SAGE lo etiqueta "[0, 1]"), y análogamente para la primera, segunda y tercera columna. Por tanto, podemos escribir</p>

In [118]:
A[[0,1],[0,1,2]]

[-1  1  1]
[-1  2  0]

<p>Para seleccionar todas las filas podemos usar el comando ":" que también nos permite seleccionar un rango "i:j" que nos proporcinará los elementos $\{i,i+1,i+2,\ldots,j-2,j-1\}$, <strong>sin alcanzar</strong> j, de esta forma podemos tomar la submatriz con las columnas segunda y tercera (sin olvidar que SAGE comienza a contar en 0)</p>

In [119]:
A[:,1:3]

[1 1]
[2 0]

In [120]:
A[:,1:3]\b

(3, -1)

<p>La solución del sistema anterior no verifica $B^{-1}b\geq 0$ por lo que no representa una solución básica factible.</p>
<p>Para que sea más sencillo el recorrido de todas las combinaciones de columnas de $A$ partimos de una lista con tantos elementos como columnas tiene $A$:</p>

In [121]:
I=range(A.ncols())

In [122]:
I

range(0, 4)

<p>La orden powerset() nos proporciona todos los subconjuntos de un conjunto, de esta forma:</p>

In [123]:
for i in powerset(I):
    i

<p>Tan sólo nos interesan los subconjuntos de dos elementos y tales que la submatriz que definan tenga rango máximo. Así</p>

In [126]:
for i in powerset(I):
    if len(i)==2 and (( (A[:,i]).det())!=0):
        print (A[:,i]\b, 'base=',i)

(2, 4) base= [0, 1]
(-6, -4) base= [0, 2]
(3, -1) base= [1, 2]
(-2, 4) base= [0, 3]
(2, 2) base= [1, 3]
(2, 6) base= [2, 3]


<p>Por ejemplo, el otuput "(2, 2)  base= [1, 3] " significa que para la matriz básica $B=[\mathbf{a_2}, \mathbf{a_4}]$, las variables básicas son $(x_2, x_4)=(2, 2)$. </p>

<p>Ahora queremos describir las direcciones extremas para ello nos bastaba construir la matriz:</p>
<p>$$\left( \!\!\begin{array}{c} - B^{-1}N \\ I \end{array} \!\!\right)$$</p>
<p>y de ésta quedarnos con las columnas positivas.<br /><br /></p>

In [15]:
for i in powerset(I):
    if (len(i)==(A.nrows())) and (((A[:,i]).det())!=0):
        noti=[]
        for l in I:
            if l not in i:
                noti.append(l)
        B=A[:,i];N=A[:,noti];Id=identity_matrix((A.ncols()-A.nrows()))
        bloque=copy(block_matrix([[-B^(-1)*N],[Id]]))
        print (bloque,'base=',i)

[ 2 -1]
[ 1 -1]
[-----]
[ 1  0]
[ 0  1] base= [0, 1]
[2 1]
[1 1]
[---]
[1 0]
[0 1] base= [0, 2]
[ 1/2 -1/2]
[ 1/2  1/2]
[---------]
[   1    0]
[   0    1] base= [1, 2]
[ 1  1]
[-1  1]
[-----]
[ 1  0]
[ 0  1] base= [0, 3]
[ 1 -1]
[-1  2]
[-----]
[ 1  0]
[ 0  1] base= [1, 3]
[ 1 -1]
[ 1 -2]
[-----]
[ 1  0]
[ 0  1] base= [2, 3]


<p>Vamos a considerar las diferentes direcciones extremas obtenidas anteriormente, ordenadas adecuadamente y añadidas a una lista. Esto es, si tomamos las direcciones correspondientes a la "base =[0,2]", (es decir, $B=[\mathbf{a}_1, \mathbf{a}_3]$), la primera columna proporcina la dirección $(d_1,d_3,d_2,d_4)^t=(2,1,1,0)^t$ y la segunda columna $(d_1,d_3,d_2,d_4)^t=(1,1,0,1)^t$. Ordenarlas, supone escribirlas como: $(d_1,d_2,d_3,d_4)^t=(2,1,1,0)^t$ y $(d_1,d_2,d_3,d_4)^t=(1,0,1,1)^t$. Por tanto la lista de direcciones debidamente ordenadas es:</p>

In [16]:
direcciones=[[2,1,1,0],[2,1,1,0],[1,0,1,1],[1,1/2,1/2,0],[1,0,1,1],[1,0,1,1]]

In [17]:
matrizdirecciones=matrix(direcciones);rank(matrizdirecciones);show(matrizdirecciones)

[  2   1   1   0]
[  2   1   1   0]
[  1   0   1   1]
[  1 1/2 1/2   0]
[  1   0   1   1]
[  1   0   1   1]

<p>Aunque obtuvimos 6 expresiones sólo tenemos dos direcciones extremas, al estudiar el rango de dicha matriz. Observemos que dicha matriz ya está traspuesta (contiene los vectores dirección por filas). Las filas 1, 2 y 3 proporcionan la mismna dirección extrema: $(2,1,1,0)^t$, e igualmenta las filas 3, 5 y 6 la segunda dirección extrema: $(1, 0, 1, 1)^t$. Ambas ordenadas según la secuencia $(d_1,d_2,d_3,d_4)$. </p>
<h3>Conos y direcciones extremas</h3>
<p>Hemos obtenido el rango de la matriz formada por todas las direcciones, pero esto en ocasiones no nos proporciona las diferentes direcciones extremas. Para entenderlo necesitasmos al menos tres dimensiones, donde podemos visualizar un cono con cuatro rayos:</p>
<p>$$\{(2,1,0),(1,2,0),(2,1,1),(1,2,1)\}$$</p>
<p>Si consideramos estas cuatro direcciones y su subespacio vectorial, tendremos dimensión:</p>

In [18]:
nuevamatrizdirecciones=matrix([[2,1,0],[1,2,0],[2,1,1],[1,2,1]])
nuevamatrizdirecciones.rank()

3

<p>Sin embargo generan un cono con cuatro direcciones extremas diferentes:</p>

In [19]:
nuevocono=Cone([[2,1,0],[1,2,0],[2,1,1],[1,2,1]])

In [20]:
nuevocono.rays()

N(2, 1, 0),
N(1, 2, 0),
N(2, 1, 1),
N(1, 2, 1)
in 3-d lattice N

<p>Repitiendo en el caso anterior, ahora con la estructura de cono, obtenemos:</p>

In [21]:
cono=Cone(direcciones)

In [22]:
cono.rays()

N(1, 0, 1, 1),
N(2, 1, 1, 0)
in 4-d lattice N

<p> </p>
<p>Si ahora estuviéramos interesados en tratar el problema:</p>
<p>$$\left. \begin{array}{rl}\min & Z(x_1, x_2)= x_1-x_2\\ \mbox{s.t.} & - x_1 + x_2 \leq 2 \\&  - x_1 + 2 x_2 \leq 6 \\&  x_1 \geq 0, x_2 \geq 0 \end{array} \right\}$$</p>
<p>Estamos interesados en saber si tenemos o no solución a dicho problema. Hemos visto en teoría que si $c^t\vec{d}\geq 0$ para toda dirección extrema $\vec{d}$, el problema tiene solución y se alcanza en un punto extremo, así</p>

In [23]:
c=vector([1,-1,0,0])

In [24]:
matrizdirecciones*c

(1, 1, 1, 1/2, 1, 1)

<p>Nuestro problema tiene solución y se alcanza en un punto extremo. Para calcular cuál, veamos cuál es el coste de los puntos extremos.</p>

In [25]:
for i in powerset(I):
    if len(i)==2 and (((A[:,i]).det())!=0):
        print (A[:,i]\b, 'base=',i)

(2, 4) base= [0, 1]
(-6, -4) base= [0, 2]
(3, -1) base= [1, 2]
(-2, 4) base= [0, 3]
(2, 2) base= [1, 3]
(2, 6) base= [2, 3]


In [26]:
extremos=[[2,4,0,0],[0,2,0,2],[0,0,2,6]]

In [27]:
matrizextremos=matrix(extremos)

<p>Para obtener el valor de la función objetivo en cada uno de los puntos extremos, multiplicamos la matriz anterior por el vector de costes.</p>

In [28]:
matrizextremos*c

(-2, -2, 0)

<p>Así se alcanza el mínimo en $(2,4)$ y en $(0,2)$, por tanto tenemos infinitas soluciones (toda la arista que va entre dichos puntos extremos). Si hubiésemos consideramos la función de costes $Z(x_1,x_2)=x_1-3x_2$</p>

In [29]:
c1=vector([1,-3,0,0])

In [30]:
matrizdirecciones*c1

(-1, -1, 1, -1/2, 1, 1)

<p>Por lo tanto no tenemos solución acotada, podemos construir una sucesión de puntos factibles $x_n$ tales que</p>
<p>$$\lim_{n\to\infty}Z(x_n)=-\infty$$</p>
<h3>Programación Lineal Entera y Mixta</h3>
<p>SAGE tiene una clase especial <tt class="descname">MixedIntegerLinearProgram</tt> que nos permite definir un problema de Programación Lineal con variables que pueden ser enteras, continuas o que haya variables de ambos tipos (en ese caso se suele hablar de Programación Mixta). Si seguimos considerando nuestro problema del principio, con función objetivo $Z(x_1,x_2)=x_1-x_2$, es decir,</p>
<p>$$\left. \begin{array}{rl}\min & Z(x_1, x_2)= x_1-x_2\\ \mbox{s.t.} & - x_1 + x_2 \leq 2 \\&  - x_1 + 2 x_2 \leq 6 \\&  x_1 \geq 0, x_2 \geq 0 \end{array} \right\}$$</p>
<p>Lo que hacemos en primer lugar es asignar a <span style="font-family: courier new,courier;"><strong>p</strong></span> la clase <tt class="descname">MixedIntegerLinearProgram</tt>, por defecto se entiende que es un problema de maximización, si estamos en un caso como el nuestro de minimización, basta añadir la opción <span class="n">maximization</span><span class="o">=</span><span class="bp">False</span>:</p>

In [31]:
p=MixedIntegerLinearProgram(maximization=False)

<p>Vamos a llamar a nuestras variables $x$ e $y$</p>

In [32]:
x, y= p['x'], p['y']

<p>Para añadir restricciones, utilizamos el método <span class="n">add_constraint</span><span class="p">(</span>)</p>

In [33]:
p.add_constraint(-x + y <= 2)
p.add_constraint(-x+2*y<=6)

<p>Para introducir la función de coste o función objetivo basta usar el método <span class="n">set_objective</span><span class="p">(</span>)</p>

In [34]:
p.set_objective(x-y)

In [35]:
p.show()

Minimization:
  x_0 - x_1 

Constraints:
  - x_0 + x_1 <= 2.0
  - x_0 + 2.0 x_1 <= 6.0
Variables:
  x_0 is a continuous variable (min=-oo, max=+oo)
  x_1 is a continuous variable (min=-oo, max=+oo)


<p>Con la versión actual de SAGE las variables por defecto no son positivas, por lo que no estamos modelando el problema de inicio.</p>
<p>Resolvemos este problema de programación lineal con el método solve(), que nos proporcionará el valor objetivo mínimo (o máximo) y si queremos conocer el valor de las variables podemos usar get_values()</p>

In [36]:
p.add_constraint(x>=0)
p.add_constraint(y>=0)
p.solve()

-2.0

In [37]:
p.get_values(x)

-0.0

In [38]:
p.get_values(y)

2.0

<p>Vamos ahora a considerar la misma región factible y cambiamos la función objetivo por $Z(x_1,x_2)=x_1-3x_2$, además introduciremos el método new_variable() que nos permite introducir un vector de variables, lo que puede ser muy útil a la hora de programar, y además nos permite añadir que dichas variables sean positivas, con la opción nonnegative=True. Aunque seguiremos la notación de clase SAGE permite el uso de cualquier nombre como variable. Por ejemplo podríamos poner "variablemia=p1.new_variable()"</p>

In [39]:
p1=MixedIntegerLinearProgram(maximization=False)

In [40]:
x=p1.new_variable(nonnegative=True)

In [41]:
p1.add_constraint(-x[0] + x[1] <= 2)
p1.add_constraint(-x[0]+2*x[1]<=6)

In [42]:
p1.set_objective(x[0]-3*x[1])

In [43]:
p1.show()

Minimization:
  x_0 -3.0 x_1 

Constraints:
  - x_0 + x_1 <= 2.0
  - x_0 + 2.0 x_1 <= 6.0
Variables:
  x_0 is a continuous variable (min=0.0, max=+oo)
  x_1 is a continuous variable (min=0.0, max=+oo)


In [44]:
p1.solve()

MIPSolverException: GLPK: The LP (relaxation) problem has no dual feasible solution

<p>Habíamos visto anteriormente que este problema no estaba acotado, por eso hemos obtenido esta respuesta. En el caso que la región factible sea vacía la respuesta de SAGE será la misma.</p>
<h3>Modelos de programación matemática</h3>
<h4>Problema de la dieta</h4>
<p>El problema por excelencia de la programación lineal es el de optimización en la asignación de recursos. Un caso concreto es el conocido como el problema de la dieta. Consiste en determinar la composición de la dieta de mínimo coste que satisface la necesidades específicas de nutrientes.</p>
<p>Consideremos el siguiente ejemplo: Se ha determinado que las necesidades mínimas diarias en la alimentación de un ternero son de 700 gr de proteínas, 28 gr de Calcio y 150 mg de vitaminas. Los alimentos disponibles son pienso y forraje con un coste unitario de 0.30 y 0.35 €/kg respectivamente. La composición nutritiva por kg de alimento se muestra en la tabla siguiente:</p>
<table style="width: 244px; height: 61px; border-color: #000000; border-width: 1px; border-style: solid;" border="1" align="center">
<tbody>
<tr>
<td> </td>
<td>
<p>Proteinas(g)</p>
</td>
<td>Calcio (g)</td>
<td>Vitaminas (mg)</td>
</tr>
<tr>
<td>Pienso</td>
<td>30</td>
<td>2</td>
<td>10</td>
</tr>
<tr>
<td>Forraje </td>
<td>45</td>
<td>1</td>
<td>5</td>
</tr>
</tbody>
</table>
<p>Para modelar este problema consideraremos una variable $x_i$ para cada tipo de variable, que sea la cantidad diaria de kg de ese alimento que se considera. Tenemos un coste $c_i$ del Kg cada uno de estos alimentos. Nuestro problema tiene como función objetivo minimizar el coste de dicha alimentación es decir $\min\sum_{i=1}^2c_ix_i$. Además tenemos restricciones dadas por las necesidades de cada uno de los nutrientes considerados, si notamos $a_{ij}$ la cantidad del nutriente $j$ en el alimento $i$ y $b_j$ la cantidad mínima diaria del nutriente $j$ dichas restricciones son una para cada tipo de nutriente y son:</p>
<p>$$\sum_i a_{ij}x_i\geq b_j$$</p>
<p>Las necesidades de nutrientes no son siempre desigualdades ($\geq$) como la anterior, pueden ser igualdades o de tipo ($\leq$).</p>
<p>En nuestro ejemplo:</p>
<p>$$\left. \begin{array}{rl}\min &  0.30 x_1+0.35 x_2\\ \mbox{s.t.} & 30x_1 + 45 x_2 \geq 700 \\&  10 x_1 +  5 x_2 \geq 150 \\ &  2 x_1 +  x_2 \geq 28 \\ &  x_1 \geq 0, x_2 \geq 0 \end{array} \right\}$$</p>

In [45]:
p2=MixedIntegerLinearProgram(maximization=False)

In [46]:
x=p2.new_variable(nonnegative=True)

In [47]:
p2.add_constraint(30*x[0] + 45*x[1] >= 700)
p2.add_constraint(10*x[0]+5*x[1]>=150)
p2.add_constraint(2*x[0]+x[1]>=28)

In [48]:
p2.set_objective(0.30*x[0]+0.35*x[1])

In [49]:
p2.solve()

6.166666666666667

In [50]:
p2.get_values(x[0])

10.833333333333337

In [51]:
p2.get_values(x[1])

8.33333333333333

También podemos pedir todas las soluciones:

In [52]:
p2.get_values(x)

{0: 10.833333333333337, 1: 8.33333333333333}

In [1]:
NUMERO_ALUMNO=17

NOMBRE_FICHERO_EXAMEN = 'Ex_PL3_MASI_t.htl'
load('codigo_examinar_html.sage')

if NUMERO_ALUMNO > 0:
    lector_examenes(NOMBRE_FICHERO_EXAMEN,NUMERO_ALUMNO,False)

1. Hay 7 puntos extremos,2. Hay 4 puntos extremos
3. Hay 3 puntos extremos,4. No hay ningún punto extremo
1. Hay 1 dirección extrema,2. Hay 3 direcciones extremas
3. No hay ninguna dirección extrema,4. Hay 4 direcciones extremas
1.,2.
3.,4. Ninguna de las otras respuestas.
"1. Sí, el nuevo coste es","2. Sí, el nuevo coste es"
3. No cambia el coste,"4. Sí, el nuevo coste es"
"1. Sí, se consumen 4.5 raciones","2. Sí, se consumen 10.0 raciones"
"3. Sí, se consumen 1.166666666666667 raciones","4. Sí, se consumen 1.9999999999999976 raciones"


In [ ]:
# EJERCICIO 1

In [ ]:
# APARTADO 1
'''
La región factible tiene como puntos extremos:
1. Hay 7 puntos extremos	2. Hay 8 puntos extremos
3. Hay 6 puntos extremos	4. Hay 3 puntos extremos
'''

In [2]:
# Convertir a forma estandar: una incognita mas por ec. (variables de holgura) y cambian los signos según la desigualdad:
'''
3x1 + 5x2 - 3x3 + x4 = 2
2x1 - 6x2 + x3 + x5 = -2
-x1 + 2x2 + 7x3 - x6 = 3
x1 + 0x2 + 4x3 + x7 = 6
'''

A=matrix([[3,5,-3,-1,0,0,0],[2,-6,1,0,1,0,0],[-1,2,7,0,0,1,0],[1,0,4,0,0,0,-1]])
b=vector([2,-2,3,6])
show(A)
show(b)

[ 3  5 -3 -1  0  0  0]
[ 2 -6  1  0  1  0  0]
[-1  2  7  0  0  1  0]
[ 1  0  4  0  0  0 -1]

(2, -2, 3, 6)

In [3]:
# Resolver el sistema:
A.solve_right(b)

(4, 7/4, 1/2, 69/4, 0, 0, 0)

In [4]:
# De otra manera:
A\b

(4, 7/4, 1/2, 69/4, 0, 0, 0)

In [5]:
# Rango de la matriz:
I=range(A.ncols())
I

# Obtener matriz cuadrada:
# A[:,0:4]

range(0, 7)

In [6]:
# Ranks:
print("Rango: ", rank(A))
print("Nº filas: ", A.nrows())
print("Nº columnas: ", A.ncols())

Rango:  4
Nº filas:  4
Nº columnas:  7


In [7]:
for i in powerset(I):
    i

In [8]:
# Subconjuntos de i elementos: solo contamos los que todo su paréntesis sea formado por nºs positivos:
'''
Es decir, aquí:
4 (30/11, 9/11, -41/11, -91/11) base= [0, 2, 3, 4]
5 (-15/4, 3/2, 101/4, -26) base= [1, 2, 3, 4]
6 (86/125, 98/125, 166/125, 897/125) base= [0, 1, 2, 5]
7 (6, 7/3, -83/3, -13/3) base= [0, 1, 3, 5]
Solo hay 1 punto extremo (el 6).
'''

j = 0
for i in powerset(I):
    if len(i)==4 and (( (A[:,i]).det())!=0):
        j = j+1
        print (j, A[:,i]\b, 'base=',i)

1 (4, 7/4, 1/2, 69/4) base= [0, 1, 2, 3]
2 (202/85, -41/85, 77/85, -897/85) base= [0, 1, 2, 4]
3 (6, 9/2, 77/2, 13) base= [0, 1, 3, 4]
4 (30/11, 9/11, 41/11, -91/11) base= [0, 2, 3, 4]
5 (-15/4, 3/2, -101/4, -26) base= [1, 2, 3, 4]
6 (86/125, 98/125, 166/125, -897/125) base= [0, 1, 2, 5]
7 (6, 7/3, 83/3, 13/3) base= [0, 1, 3, 5]
8 (-2, 2, -14, -13) base= [0, 2, 3, 5]
9 (7/12, 3/2, -43/12, -26/3) base= [1, 2, 3, 5]
10 (6, -16/5, -166/5, 77/5) base= [0, 1, 4, 5]
11 (26/15, 16/15, -98/15, -41/15) base= [0, 2, 4, 5]
12 (13/10, 3/2, 43/10, -101/10) base= [1, 2, 4, 5]
13 (6, 16, -14, 9) base= [0, 3, 4, 5]
14 (3/2, -13/2, -7/2, -15/2) base= [2, 3, 4, 5]
15 (15/67, 31/67, 22/67, -299/67) base= [0, 1, 2, 6]
16 (-7, -2, -33, -13) base= [0, 1, 3, 6]
17 (-17/15, 4/15, -31/5, -91/15) base= [0, 2, 3, 6]
18 (17/44, 7/22, -45/44, -52/11) base= [1, 2, 3, 6]
19 (-1, 1, 6, -7) base= [0, 1, 4, 6]
20 (23/18, 11/18, -31/6, -41/18) base= [0, 2, 4, 6]
21 (23/41, 11/41, 45/41, -202/41) base= [1, 2, 4, 6]
22 (-

In [ ]:
# APARTADO 2
'''
La región factible tiene como direcciones extremas:
1. Hay 2 direcciones extremas	2. Hay 5 direcciones extremas
3. No hay ninguna dirección extrema	4. Hay 3 direcciones extremas
'''

In [9]:
'''
De cada matriz, formado por dos partes, la superior de términos y la inferior es la identidad. Observar las columnas!. Por cada
matriz, descartar TODAS las columnas que contengan algún término negativo. Si encontramos una columna que contenga todos sus
elementos positivos, la contamos PERO si aparece otra columna IGUAL, representa el mismo vector y no se sumaría al contador.
De forma que aquí:
[ -24/125    -4/25  -13/125]
[  -7/125     3/25   -9/125]
[   6/125     1/25  -28/125]
[  52/125    17/25 -201/125]
[--------------------------]
[       1        0        0]
[       0        1        0]
[       0        0        1] base= [0, 1, 2, 5]
Por ejemplo, ninguna columna es definida positiva así que habría 0 direcciones extremas.
En otro caso, si por ejemplo tenemos:
[  8/11  4/11  4/11]
[ 2/11   1/11  1/11]
[ 85/11  15/11  15/11]
[ 52/11   7/11  7/11]
[--------------------]
[     1      0      0]
[     0      1      0]
[     0      0      1] base= [0, 2, 3, 4]
Tendríamos 2 direcciones extremas porque una de ellas se repite. 

La base nos da el orden para representar los elementos de cada columna.
'''

for i in powerset(I):
    if (len(i)==(A.nrows())) and (((A[:,i]).det())!=0):
        noti=[]
        for l in I:
            if l not in i:
                noti.append(l)
        B=A[:,i];N=A[:,noti];Id=identity_matrix((A.ncols()-A.nrows()))
        bloque=copy(block_matrix([[-B^(-1)*N],[Id]]))
        print (bloque,'base=',i)

[  2/13   6/13  11/13]
[ 11/52   7/52  15/52]
[ -1/26  -3/26   1/26]
[ 85/52 125/52 201/52]
[--------------------]
[     1      0      0]
[     0      1      0]
[     0      0      1] base= [0, 1, 2, 3]
[   8/85    4/17   41/85]
[  11/85   -3/17  -18/85]
[  -2/85   -1/17   11/85]
[  52/85  -25/17 -201/85]
[-----------------------]
[      1       0       0]
[      0       1       0]
[      0       0       1] base= [0, 1, 2, 4]
[   -4     0     1]
[-11/2  -1/2   1/2]
[-85/2  -5/2  11/2]
[  -26    -3     1]
[-----------------]
[    1     0     0]
[    0     1     0]
[    0     0     1] base= [0, 1, 3, 4]
[  8/11   4/11   7/11]
[ -2/11  -1/11   1/11]
[ 85/11  15/11  18/11]
[ 52/11  -7/11 -15/11]
[--------------------]
[     1      0      0]
[     0      1      0]
[     0      0      1] base= [0, 2, 3, 4]
[ 11/8  -1/2  -7/8]
[ -1/4     0   1/4]
[ 85/8  -5/2 -41/8]
[ 13/2    -3 -11/2]
[-----------------]
[    1     0     0]
[    0     1     0]
[    0     0     1] base= [1, 2, 3, 4]
[  24/125

In [ ]:
# APARTADO 3
'''
Si queremos  minx1−x2+x3  en nuestra región  S .¿Existe solución óptima?
1. Existe solución óptima única  (x1,x2,x3)=(0,712,32) 
2. Existe solución óptima única  (x1,x2,x3)=(0,25,0) 
3. Existe solución óptima única  (x1,x2,x3)=(0,2341,1141) 
4. Ninguna de las otras respuestas
'''

In [ ]:
# EJERCICIO 2

In [ ]:
# APARTADO 1
'''
1.  Z=304.0 	2.  Z=341.6 
3.  Z=310.66666666666663 	4. Ninguna de las otras respuestas.
'''

# APARTADO 2
'''
1. Alimento 2: 10.6 y Alimento 6: 7.8
2. Alimento 5: 9.666666666666666 y Alimento 2: 6.666666666666667
3. Alimento 5: 11.0 y Alimento 2: 3.9999999999999996
4. Alimento 5: 10.333333333333334 y Alimento 3: 9.333333333333332
'''

In [10]:
p=MixedIntegerLinearProgram(maximization=False) #SIEMPRE
x=p.new_variable(real=True, nonnegative=True) # SIEMPRE

#A MODELAR:
'''
Cada línea es una restriccion: representan:
    Cantidad * [elemento1] + cantidad * [elemento2] + .....
    Si por ejemplo, para la vitamina A se pide:
        VitA 1 1 0 3 2 1
    Pues se escribe como:
        1*x[1]+1*x[2]+0*x[3]+3*x[4]+2*x[5]+1*x[6]>=X
    Si dice "a lo sumo", implica <= X
    Si dice "al menos", implica >= X
    Si pide maximizar en lugar de minimizar, se cambia el valor maximization (arriba) por True.
    El valor objetivo resuelve el apartado 1.
    El campo solucion muestra la solucion de todas las variables y resuelve el apartado 2.
'''

# Restricciones:
p.add_constraint(1*x[1]+1*x[2]+0*x[3]+2*x[4]+2*x[5]+1*x[6]>=26)
p.add_constraint(1*x[1]+1*x[2]+0*x[3]+2*x[4]+2*x[5]+1*x[6]<=32) # TIENE QUE ESTAR ENTRE 26 Y 32 ASÍ QUE DOBLO LA RESTRICCIÓN
p.add_constraint(1*x[1]+2*x[2]+1*x[3]+1*x[4]+1*x[5]+0*x[6]>=25)
p.add_constraint(0*x[1]+1*x[2]+1*x[3]+0*x[4]+1*x[5]+2*x[6]==30)
p.add_constraint(1*x[1]+0*x[2]+0*x[3]+1*x[4]+0*x[5]+1*x[6]<=14)

# Funcion objetivo:
p.set_objective(10*x[1]+14*x[2]+10*x[3]+15*x[4]+18*x[5]+16*x[6])

print ('Valor objetivo=', p.solve()) #SIEMPRE
print ('Solución=', p.get_values(x)) #SIEMPRE

Valor objetivo= 341.6
Solución= {1: 0.0, 2: 10.6, 3: 0.0, 4: 0.0, 5: 3.799999999999999, 6: 7.800000000000002}


In [ ]:
# APARTADO 3
'''
1. Sí, el nuevo coste es  Z=346.4285714285715 	2. No cambia el coste
3. Sí, el nuevo coste es  Z=311.0 	4. Sí, el nuevo coste es  Z=324.16666666666663
'''

In [11]:
p=MixedIntegerLinearProgram(maximization=False) #SIEMPRE
x=p.new_variable(real=True, nonnegative=True) # SIEMPRE

# Restricciones:
p.add_constraint(1*x[1]+1*x[2]+0*x[3]+2*x[4]+2*x[5]+1*x[6]>=26)
p.add_constraint(1*x[1]+1*x[2]+0*x[3]+2*x[4]+2*x[5]+1*x[6]<=32) # TIENE QUE ESTAR ENTRE 26 Y 32 ASÍ QUE DOBLO LA RESTRICCIÓN
p.add_constraint(1*x[1]+2*x[2]+1*x[3]+1*x[4]+1*x[5]+0*x[6]>=25)
p.add_constraint(0*x[1]+1*x[2]+1*x[3]+0*x[4]+1*x[5]+2*x[6]==30)
p.add_constraint(1*x[1]+0*x[2]+0*x[3]+1*x[4]+0*x[5]+1*x[6]>=5) # la condición que se cambia del apartado 3(5mg de vitamina D)

# Funcion objetivo:
p.set_objective(10*x[1]+14*x[2]+10*x[3]+15*x[4]+18*x[5]+16*x[6])

'''
El resultado del apartado 3 viene dado por el valor objetivo solo.
'''
print ('Valor objetivo=', p.solve()) #SIEMPRE
print ('Solución=', p.get_values(x)) #SIEMPRE

Valor objetivo= 341.6
Solución= {1: 0.0, 2: 10.6, 3: 0.0, 4: 0.0, 5: 3.8000000000000003, 6: 7.8}


In [ ]:
# APARTADO 4
'''
1. Sí, se consumen 10.0 raciones	2. Sí, se consumen 4.5 raciones
3. Sí, se consumen 1.9999999999999976 raciones	4. No se consume el alimento 1
'''

In [16]:
p=MixedIntegerLinearProgram(maximization=False) #SIEMPRE
x=p.new_variable(real=True, nonnegative=True) # SIEMPRE

# Restricciones:
p.add_constraint(1*x[1]+1*x[2]+0*x[3]+2*x[4]+2*x[5]+1*x[6]>=26)
p.add_constraint(1*x[1]+1*x[2]+0*x[3]+2*x[4]+2*x[5]+1*x[6]<=32) # TIENE QUE ESTAR ENTRE 26 Y 32 ASÍ QUE DOBLO LA RESTRICCIÓN
p.add_constraint(1*x[1]+2*x[2]+1*x[3]+1*x[4]+1*x[5]+0*x[6]>=25)
p.add_constraint(0*x[1]+1*x[2]+1*x[3]+0*x[4]+1*x[5]+2*x[6]==30)
# PROBAR EL VALOR DE LA VITAMINA DEL APARTADO ANTERIOR Y DEL ENUNCIADO. QUEDARSE CON LA SOLUCIÓN BUENA.
# Esta penultima restriccion cambia, puede tener el valor del apartado 1 o del 2. Probar los dos.
p.add_constraint(1*x[1]+0*x[2]+0*x[3]+1*x[4]+0*x[5]+1*x[6]<=14)
# p.add_constraint(1*x[1]+0*x[2]+0*x[3]+1*x[4]+0*x[5]+1*x[6]>=5)

# Esta es la restriccion nueva del apartado 4. 
p.add_constraint(x[1]+x[2]+x[3]+x[4]+x[5]+x[6]>=25)

# Funcion objetivo:
p.set_objective(10*x[1]+14*x[2]+10*x[3]+15*x[4]+18*x[5]+16*x[6])

'''
El resultado se obtiene de la solucion p.get_values().
'''
print ('Valor objetivo=', p.solve()) #SIEMPRE
print ('Solución=', p.get_values(x)) #SIEMPRE

Valor objetivo= 355.0
Solución= {1: 4.5, 2: 10.0, 3: 0.0, 4: 0.0, 5: 0.9999999999999997, 6: 9.5}
